## Классификатор языков 

In [ ]:
#идея: допустим, что всех слов языка у нас нет. каждый язык будет иметь наборы последовательностей из 3х букв. 
#и по поиску этих последовательнойстей можно определять язык. можно составить матирцу корреляции и выкинуть сильно 
#коррелирующие значения, чтобы было меньше ошибок
#но это сложная идея из-за предварительной обработки датасета (разделение на последовательности из 3х букв)

In [ ]:
#для начала будем пробовать собрать инфу из всех 3х словарей в один словарь и для каждого слова тэгом пометить 
#к какому языку он отностися.

#нужна сверточная сеть, так как идея та же. пробегаемся некоторым окном по большим данным

In [27]:
data = []
labels = []
langs = ['dutch.txt', 'hungarian.txt', 'portugese.txt']
for filename in langs:
    with open(filename, 'r') as input_file:
        for line in input_file:
            data.append(line.strip().lower())
            labels.append(langs.index(filename))
print(data[:10])
print(labels[:10])

ALPHABET = sorted(set(''.join(data)))
print(ALPHABET)

['ik', 'je', 'het', 'de', 'dat', 'is', 'een', 'niet', 'en', 'wat']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
["'", '-', '.', '7', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'á', 'â', 'ã', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'ï', 'ó', 'ô', 'õ', 'ö', 'ú', 'ü', 'ő', 'ű']


In [40]:
max_len = 0
for word in data:
    if len(word) > max_len:
        max_len = len(word)
print(max_len)

20


In [28]:
un = list(set(data))

In [29]:
# for i, word in enumerate(data):
#     data[i] = un.index(word)
# print(data[:10])

In [32]:
#нужно преобразовать строки в формат, который пригоден для передачи на обучение. попробуем сделать np.array из строк

In [41]:
import numpy as np

MAX_LEN = 20 #максимальная длина слова

def encode_sample(sample, index):
    indices = [index[char] for char in sample if char in index]     
    return np.resize(np.array(indices), MAX_LEN)

index = {char: i + 1 for i, char in enumerate(ALPHABET)}
print(index)
X = np.stack([encode_sample(x.lower(), index) for x in data])#преобразовываем слово в список чисел, соотв. буквам по нашему алфавиту
print(X[:10])
y = np.eye(len(langs))[labels]#преобразовываем в вектор
print(y[:10])

{"'": 1, '-': 2, '.': 3, '7': 4, 'a': 5, 'b': 6, 'c': 7, 'd': 8, 'e': 9, 'f': 10, 'g': 11, 'h': 12, 'i': 13, 'j': 14, 'k': 15, 'l': 16, 'm': 17, 'n': 18, 'o': 19, 'p': 20, 'q': 21, 'r': 22, 's': 23, 't': 24, 'u': 25, 'v': 26, 'w': 27, 'x': 28, 'y': 29, 'z': 30, 'á': 31, 'â': 32, 'ã': 33, 'ç': 34, 'è': 35, 'é': 36, 'ê': 37, 'ë': 38, 'í': 39, 'ï': 40, 'ó': 41, 'ô': 42, 'õ': 43, 'ö': 44, 'ú': 45, 'ü': 46, 'ő': 47, 'ű': 48}
[[13 15 13 15 13 15 13 15 13 15 13 15 13 15 13 15 13 15 13 15]
 [14  9 14  9 14  9 14  9 14  9 14  9 14  9 14  9 14  9 14  9]
 [12  9 24 12  9 24 12  9 24 12  9 24 12  9 24 12  9 24 12  9]
 [ 8  9  8  9  8  9  8  9  8  9  8  9  8  9  8  9  8  9  8  9]
 [ 8  5 24  8  5 24  8  5 24  8  5 24  8  5 24  8  5 24  8  5]
 [13 23 13 23 13 23 13 23 13 23 13 23 13 23 13 23 13 23 13 23]
 [ 9  9 18  9  9 18  9  9 18  9  9 18  9  9 18  9  9 18  9  9]
 [18 13  9 24 18 13  9 24 18 13  9 24 18 13  9 24 18 13  9 24]
 [ 9 18  9 18  9 18  9 18  9 18  9 18  9 18  9 18  9 18  9 18]
 [27  5 2

In [44]:
from keras import layers, models, optimizers

LAYER_PARAMS = [[64, 3, 3], [128, 3, 3]]
EMBEDDING_DIM = 16

def build_model():
    inputs = layers.Input(shape=(MAX_LEN,))
    x = layers.Embedding(len(ALPHABET)+1, EMBEDDING_DIM)(inputs)
    for filters, kernel_size, pool_size in LAYER_PARAMS:
        x = layers.Conv1D(filters, kernel_size, activation="relu")(x)
        x = layers.BatchNormalization()(x)
        x = layers.SpatialDropout1D(0.15)(x)
        x = layers.MaxPooling1D(pool_size)(x)
        
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(len(langs), activation="softmax")(x)
    
    model = models.Model(inputs=inputs, outputs=x)
    model.compile(optimizer=optimizers.Adadelta(),
                  loss="categorical_crossentropy", metrics=["acc"])
    return model

model = build_model()
history = model.fit(X, y, epochs=500, batch_size=20, 
                    validation_split=0.2)

Epoch 1/500
568/568 [==============================] - 2s 2ms/step - loss: 1.8229 - acc: 0.3639 - val_loss: 1.0168 - val_acc: 0.4852
Epoch 2/500
568/568 [==============================] - 1s 2ms/step - loss: 1.7645 - acc: 0.3784 - val_loss: 2.3639 - val_acc: 0.1551
Epoch 3/500
568/568 [==============================] - 1s 2ms/step - loss: 1.7149 - acc: 0.3899 - val_loss: 2.3094 - val_acc: 0.1636
Epoch 4/500
568/568 [==============================] - 1s 2ms/step - loss: 1.6723 - acc: 0.3917 - val_loss: 2.2171 - val_acc: 0.1735
Epoch 5/500
568/568 [==============================] - 1s 2ms/step - loss: 1.6339 - acc: 0.3980 - val_loss: 2.1441 - val_acc: 0.1823
Epoch 6/500
568/568 [==============================] - 1s 2ms/step - loss: 1.5838 - acc: 0.4028 - val_loss: 2.1117 - val_acc: 0.1876
Epoch 7/500
568/568 [==============================] - 1s 2ms/step - loss: 1.5619 - acc: 0.4140 - val_loss: 2.0814 - val_acc: 0.1932
Epoch 8/500
568/568 [==============================] - 1s 2ms/step - 

568/568 [==============================] - 1s 2ms/step - loss: 0.9785 - acc: 0.5909 - val_loss: 1.0509 - val_acc: 0.5339
Epoch 63/500
568/568 [==============================] - 1s 2ms/step - loss: 0.9685 - acc: 0.5935 - val_loss: 1.0471 - val_acc: 0.5360
Epoch 64/500
568/568 [==============================] - 1s 2ms/step - loss: 0.9781 - acc: 0.5880 - val_loss: 1.0474 - val_acc: 0.5367
Epoch 65/500
568/568 [==============================] - 1s 2ms/step - loss: 0.9600 - acc: 0.5928 - val_loss: 1.0219 - val_acc: 0.5472
Epoch 66/500
568/568 [==============================] - 1s 2ms/step - loss: 0.9863 - acc: 0.5892 - val_loss: 1.0358 - val_acc: 0.5398
Epoch 67/500
568/568 [==============================] - 1s 2ms/step - loss: 0.9579 - acc: 0.5983 - val_loss: 1.0283 - val_acc: 0.5437
Epoch 68/500
568/568 [==============================] - 1s 2ms/step - loss: 0.9613 - acc: 0.5943 - val_loss: 1.0173 - val_acc: 0.5490
Epoch 69/500
568/568 [==============================] - 1s 2ms/step - loss:

568/568 [==============================] - 1s 2ms/step - loss: 0.8182 - acc: 0.6566 - val_loss: 0.8647 - val_acc: 0.6305
Epoch 124/500
568/568 [==============================] - 1s 2ms/step - loss: 0.7998 - acc: 0.6689 - val_loss: 0.8463 - val_acc: 0.6379
Epoch 125/500
568/568 [==============================] - 1s 2ms/step - loss: 0.8096 - acc: 0.6640 - val_loss: 0.8658 - val_acc: 0.6291
Epoch 126/500
568/568 [==============================] - 1s 2ms/step - loss: 0.8012 - acc: 0.6707 - val_loss: 0.8385 - val_acc: 0.6425
Epoch 127/500
568/568 [==============================] - 1s 2ms/step - loss: 0.8003 - acc: 0.6680 - val_loss: 0.8169 - val_acc: 0.6509
Epoch 128/500
568/568 [==============================] - 1s 2ms/step - loss: 0.7984 - acc: 0.6651 - val_loss: 0.8295 - val_acc: 0.6492
Epoch 129/500
568/568 [==============================] - 1s 2ms/step - loss: 0.8004 - acc: 0.6695 - val_loss: 0.8333 - val_acc: 0.6442
Epoch 130/500
568/568 [==============================] - 1s 2ms/step 

Epoch 184/500
568/568 [==============================] - 1s 2ms/step - loss: 0.7291 - acc: 0.7000 - val_loss: 0.7716 - val_acc: 0.6700
Epoch 185/500
568/568 [==============================] - 1s 2ms/step - loss: 0.7235 - acc: 0.6963 - val_loss: 0.7725 - val_acc: 0.6703
Epoch 186/500
568/568 [==============================] - 1s 2ms/step - loss: 0.7192 - acc: 0.6998 - val_loss: 0.7697 - val_acc: 0.6717
Epoch 187/500
568/568 [==============================] - 1s 2ms/step - loss: 0.7124 - acc: 0.7011 - val_loss: 0.7408 - val_acc: 0.6841
Epoch 188/500
568/568 [==============================] - 1s 2ms/step - loss: 0.7166 - acc: 0.6974 - val_loss: 0.7424 - val_acc: 0.6809
Epoch 189/500
568/568 [==============================] - 1s 2ms/step - loss: 0.7114 - acc: 0.7006 - val_loss: 0.7550 - val_acc: 0.6784
Epoch 190/500
568/568 [==============================] - 1s 2ms/step - loss: 0.7040 - acc: 0.7105 - val_loss: 0.7589 - val_acc: 0.6760
Epoch 191/500
568/568 [==============================] 

568/568 [==============================] - 1s 2ms/step - loss: 0.6508 - acc: 0.7294 - val_loss: 0.6817 - val_acc: 0.7123
Epoch 245/500
568/568 [==============================] - 1s 2ms/step - loss: 0.6415 - acc: 0.7310 - val_loss: 0.7006 - val_acc: 0.7031
Epoch 246/500
568/568 [==============================] - 1s 2ms/step - loss: 0.6511 - acc: 0.7287 - val_loss: 0.6823 - val_acc: 0.7133
Epoch 247/500
568/568 [==============================] - 1s 2ms/step - loss: 0.6525 - acc: 0.7286 - val_loss: 0.6960 - val_acc: 0.7087
Epoch 248/500
568/568 [==============================] - 1s 2ms/step - loss: 0.6473 - acc: 0.7296 - val_loss: 0.6646 - val_acc: 0.7200
Epoch 249/500
568/568 [==============================] - 1s 2ms/step - loss: 0.6429 - acc: 0.7311 - val_loss: 0.6828 - val_acc: 0.7087
Epoch 250/500
568/568 [==============================] - 1s 2ms/step - loss: 0.6502 - acc: 0.7305 - val_loss: 0.6703 - val_acc: 0.7154
Epoch 251/500
568/568 [==============================] - 1s 2ms/step 

Epoch 305/500
568/568 [==============================] - 1s 2ms/step - loss: 0.6078 - acc: 0.7446 - val_loss: 0.6234 - val_acc: 0.7408
Epoch 306/500
568/568 [==============================] - 1s 2ms/step - loss: 0.6051 - acc: 0.7484 - val_loss: 0.6518 - val_acc: 0.7236
Epoch 307/500
568/568 [==============================] - 1s 2ms/step - loss: 0.6023 - acc: 0.7470 - val_loss: 0.6432 - val_acc: 0.7306
Epoch 308/500
568/568 [==============================] - 1s 2ms/step - loss: 0.6025 - acc: 0.7477 - val_loss: 0.6190 - val_acc: 0.7429
Epoch 309/500
568/568 [==============================] - 1s 2ms/step - loss: 0.6116 - acc: 0.7494 - val_loss: 0.6363 - val_acc: 0.7317
Epoch 310/500
568/568 [==============================] - 1s 2ms/step - loss: 0.6047 - acc: 0.7518 - val_loss: 0.6284 - val_acc: 0.7391
Epoch 311/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5992 - acc: 0.7504 - val_loss: 0.6287 - val_acc: 0.7387
Epoch 312/500
568/568 [==============================] 

568/568 [==============================] - 1s 2ms/step - loss: 0.5704 - acc: 0.7640 - val_loss: 0.6089 - val_acc: 0.7461
Epoch 366/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5617 - acc: 0.7694 - val_loss: 0.6052 - val_acc: 0.7468
Epoch 367/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5638 - acc: 0.7685 - val_loss: 0.5997 - val_acc: 0.7532
Epoch 368/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5705 - acc: 0.7660 - val_loss: 0.6045 - val_acc: 0.7504
Epoch 369/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5710 - acc: 0.7590 - val_loss: 0.6079 - val_acc: 0.7472
Epoch 370/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5581 - acc: 0.7647 - val_loss: 0.6110 - val_acc: 0.7454
Epoch 371/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5652 - acc: 0.7674 - val_loss: 0.5880 - val_acc: 0.7563
Epoch 372/500
568/568 [==============================] - 1s 2ms/step 

Epoch 426/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5347 - acc: 0.7791 - val_loss: 0.5571 - val_acc: 0.7722
Epoch 427/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5446 - acc: 0.7784 - val_loss: 0.5847 - val_acc: 0.7581
Epoch 428/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5373 - acc: 0.7846 - val_loss: 0.5649 - val_acc: 0.7666
Epoch 429/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5442 - acc: 0.7777 - val_loss: 0.5676 - val_acc: 0.7669
Epoch 430/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5359 - acc: 0.7793 - val_loss: 0.5760 - val_acc: 0.7609
Epoch 431/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5373 - acc: 0.7720 - val_loss: 0.5472 - val_acc: 0.7789
Epoch 432/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5431 - acc: 0.7741 - val_loss: 0.5740 - val_acc: 0.7616
Epoch 433/500
568/568 [==============================] 

568/568 [==============================] - 1s 2ms/step - loss: 0.5145 - acc: 0.7904 - val_loss: 0.5505 - val_acc: 0.7715
Epoch 487/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5108 - acc: 0.7880 - val_loss: 0.5540 - val_acc: 0.7715
Epoch 488/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5073 - acc: 0.7907 - val_loss: 0.5202 - val_acc: 0.7898
Epoch 489/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5165 - acc: 0.7864 - val_loss: 0.5520 - val_acc: 0.7754
Epoch 490/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5106 - acc: 0.7937 - val_loss: 0.5409 - val_acc: 0.7807
Epoch 491/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5001 - acc: 0.7955 - val_loss: 0.5372 - val_acc: 0.7821
Epoch 492/500
568/568 [==============================] - 1s 2ms/step - loss: 0.5025 - acc: 0.7955 - val_loss: 0.5542 - val_acc: 0.7726
Epoch 493/500
568/568 [==============================] - 1s 2ms/step 